This is a companion notebook for the book [Deep Learning with Python, Second Edition](https://www.manning.com/books/deep-learning-with-python-second-edition?a_aid=keras&a_bid=76564dff). For readability, it only contains runnable code blocks and section titles, and omits everything else in the book: text paragraphs, figures, and pseudocode.

**If you want to be able to follow what's going on, I recommend reading the notebook side by side with your copy of the book.**

This notebook was generated for TensorFlow 2.6.

# Best practices for the real world

## Getting the most out of your models

### Hyperparameter optimization

#### Using KerasTuner

In [ ]:
!pip install keras-tuner -q # installing keras-tuner library for hyperparameter tuning 

**A KerasTuner model-building function**

In [ ]:
from tensorflow import keras # importing keras library from tensorflow
from tensorflow.keras import layers # importing layers from keras

def build_model(hp): # defining a function build_model with hp as parameter
    units = hp.Int(name="units", min_value=16, max_value=64, step=16) # defining units as hyperparameter with min_value=16, max_value=64, step=16
    model = keras.Sequential([ # defining a sequential model
        layers.Dense(units, activation="relu"), # adding a dense layer with units and relu activation function
        layers.Dense(10, activation="softmax") # adding a dense layer with 10 units and softmax activation function
    ])
    optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"]) # defining optimizer as hyperparameter with values as rmsprop and adam
    model.compile( # compiling the model
        optimizer=optimizer, # using optimizer as optimizer
        loss="sparse_categorical_crossentropy", # using sparse_categorical_crossentropy as loss function
        metrics=["accuracy"]) # using accuracy as metric
    return model # returning the model

**A KerasTuner `HyperModel`**

In [ ]:
import kerastuner as kt # importing kerastuner library

class SimpleMLP(kt.HyperModel): # defining a class SimpleMLP with kt.HyperModel as parent class
    def __init__(self, num_classes): # defining a constructor with self and num_classes as parameters
        self.num_classes = num_classes # initializing num_classes with num_classes

    def build(self, hp): # defining a function build with self and hp as parameters
        units = hp.Int(name="units", min_value=16, max_value=64, step=16) # defining units as hyperparameter with min_value=16, max_value=64, step=16
        model = keras.Sequential([ # defining a sequential model
            layers.Dense(units, activation="relu"), # adding a dense layer with units and relu activation function
            layers.Dense(self.num_classes, activation="softmax") # adding a dense layer with num_classes and softmax activation function
        ])
        optimizer = hp.Choice(name="optimizer", values=["rmsprop", "adam"]) # defining optimizer as hyperparameter with values as rmsprop and adam
        model.compile( # compiling the model
            optimizer=optimizer, # using optimizer as optimizer
            loss="sparse_categorical_crossentropy", # using sparse_categorical_crossentropy as loss function
            metrics=["accuracy"]) # using accuracy as metric
        return model # returning the model

hypermodel = SimpleMLP(num_classes=10) # creating an object hypermodel of class SimpleMLP with num_classes=10 as parameter 

In [ ]:
tuner = kt.BayesianOptimization( # defining a BayesianOptimization tuner
    build_model, # using build_model function
    objective="val_accuracy", # using val_accuracy as objective
    max_trials=100, # using max_trials as 100
    executions_per_trial=2, # using executions_per_trial as 2
    directory="mnist_kt_test", # using directory as mnist_kt_test
    overwrite=True, # using overwrite as True
)

In [ ]:
tuner.search_space_summary() # printing the search space summary

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data() # loading mnist dataset
x_train = x_train.reshape((-1, 28 * 28)).astype("float32") / 255 # reshaping x_train and converting it to float32
x_test = x_test.reshape((-1, 28 * 28)).astype("float32") / 255 # reshaping x_test and converting it to float32
x_train_full = x_train[:] # copying x_train to x_train_full
y_train_full = y_train[:] # copying y_train to y_train_full
num_val_samples = 10000 # defining num_val_samples as 10000
x_train, x_val = x_train[:-num_val_samples], x_train[-num_val_samples:] # splitting x_train into x_train and x_val
y_train, y_val = y_train[:-num_val_samples], y_train[-num_val_samples:] # splitting y_train into y_train and y_val
callbacks = [ # defining a list of callbacks
    keras.callbacks.EarlyStopping(monitor="val_loss", patience=5), # adding EarlyStopping callback with monitor as val_loss and patience as 5 
]
tuner.search( # searching the tuner
    x_train, y_train, # using x_train and y_train as parameters 
    batch_size=128, # using batch_size as 128
    epochs=100, # using epochs as 100
    validation_data=(x_val, y_val), # using x_val and y_val as validation data
    callbacks=callbacks, # using callbacks as callbacks
    verbose=2, # using verbose as 2
)

**Querying the best hyperparameter configurations**

In [ ]:
top_n = 4 # defining top_n as 4
best_hps = tuner.get_best_hyperparameters(top_n) # getting the best hyperparameters

In [ ]:
def get_best_epoch(hp): # defining a function get_best_epoch with hp as parameter
    model = build_model(hp) # building the model using hp
    callbacks=[ # defining a list of callbacks
        keras.callbacks.EarlyStopping( # adding EarlyStopping callback
            monitor="val_loss", mode="min", patience=10) # with monitor as val_loss, mode as min and patience as 10
    ] 
    history = model.fit( # fitting the model
        x_train, y_train, # using x_train and y_train as parameters
        validation_data=(x_val, y_val), # using x_val and y_val as validation data
        epochs=100, # using epochs as 100
        batch_size=128, # using batch_size as 128
        callbacks=callbacks) # using callbacks as callbacks
    val_loss_per_epoch = history.history["val_loss"] # getting val_loss per epoch
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1 # getting the best epoch
    print(f"Best epoch: {best_epoch}") # printing the best epoch
    return best_epoch # returning the best epoch

In [ ]:
def get_best_trained_model(hp): # defining a function get_best_trained_model with hp as parameter
    best_epoch = get_best_epoch(hp) # getting the best epoch
    model = build_model(hp) # building the model using hp
    model.fit( # fitting the model
        x_train_full, y_train_full, # using x_train_full and y_train_full as parameters
        batch_size=128, epochs=int(best_epoch * 1.2)) # using batch_size as 128 and epochs as best_epoch * 1.2
    return model # returning the model

best_models = [] # defining a list best_models
for hp in best_hps: # iterating through best_hps 
    model = get_best_trained_model(hp) # getting the best trained model
    model.evaluate(x_test, y_test) # evaluating the model
    best_models.append(model) # appending the model to best_models

In [ ]:
best_models = tuner.get_best_models(top_n) # getting the best models

#### The art of crafting the right search space

#### The future of hyperparameter tuning: automated machine learning

### Model ensembling

## Scaling-up model training

### Speeding up training on GPU with mixed precision

#### Understanding floating-point precision

In [ ]:
import tensorflow as tf # importing tensorflow library
import numpy as np # importing numpy library
np_array = np.zeros((2, 2)) # creating a numpy array of shape (2, 2)
tf_tensor = tf.convert_to_tensor(np_array) # converting numpy array to tensor
tf_tensor.dtype # getting the data type of tensor

In [ ]:
np_array = np.zeros((2, 2)) # creating a numpy array of shape (2, 2)
tf_tensor = tf.convert_to_tensor(np_array, dtype="float32") # converting numpy array to tensor with data type as float32
tf_tensor.dtype # getting the data type of tensor

#### Mixed-precision training in practice

In [ ]:
from tensorflow import keras # importing keras library from tensorflow
keras.mixed_precision.set_global_policy("mixed_float16") # setting the global policy to mixed_float16

### Multi-GPU training

#### Getting your hands on two or more GPUs

#### Single-host, multi-device synchronous training

### TPU training

#### Using a TPU via Google Colab

#### Leveraging step fusing to improve TPU utilization

## Summary